In [24]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import k3d
import numpy as np
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
from bmcs_shell.api import WBTessellation4PEx
from scipy.spatial import cKDTree
import pandas as pd
from openpyxl import load_workbook
import k3d
import numpy as np
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
from bmcs_shell.api import WBTessellation4PEx
from scipy.spatial import cKDTree
import pandas as pd
from openpyxl import load_workbook

# Set the reference geometry to be generated from the closed form kinematics

wb_shell = WBTessellation4PEx(
                         a=1000/4,
                         b = 1615/4, 
                         c = 645/4, 
                         e_x = 286/4,
                         gamma=0.683, # a value of gamma = 0.75 was estimated from normals, but a CAD comparison showed that 0.75 doesn't lead to closer geometry to the scanned 
                         n_phi_plus=2, # planned 5 
                         n_x_plus=2,  # planned 3
                         wireframe_width=5,
                        ##---- Trimming function works only in WBTessellation4P ----##
                         trim_half_cells_along_y=True,
                         trim_half_cells_along_x=True,
#                          align_outer_nodes_along_x=True,
)
# wb_shell.interact()
orig_I_Fi = np.copy(wb_shell.I_Fi_trimmed)
orig_X_Ia = np.copy(wb_shell.X_Ia_trimmed)

orig_X_Ia = orig_X_Ia
orig_X_Ib = orig_X_Ia
orig_X_Ib, orig_I_Fi


wb_cell_default - WBTessellation4PEx
update_wb_cell_params - WBTessellation4PEx
delta_x - WBCell4ParamEx
X_Ia - WBCell4ParamEx
I_Fi - WBCell4ParamEx
idx_of_facets_to_trim - WBTessellation4PEx
X_Ia_no_constraint - WBTessellation4PEx


(array([[ 7.15000000e+01,  0.00000000e+00,  0.00000000e+00],
        [-7.15000000e+01,  0.00000000e+00,  0.00000000e+00],
        [ 3.21500000e+02,  3.85849365e+02,  1.18888672e+02],
        [-3.21500000e+02,  3.85849365e+02,  1.18888672e+02],
        [ 3.21500000e+02, -3.85849365e+02,  1.18888672e+02],
        [-3.21500000e+02, -3.85849365e+02,  1.18888672e+02],
        [ 1.73268570e+02,  0.00000000e+00,  1.25079102e+02],
        [-1.73268570e+02,  0.00000000e+00,  1.25079102e+02],
        [-4.23268569e+02,  3.81836609e+02, -6.12597656e+00],
        [-4.94768569e+02,  3.81836609e+02, -6.12597656e+00],
        [-4.94768554e+02, -7.62939453e-06,  1.25079102e+02],
        [-4.23268569e+02, -3.81836609e+02, -6.12597656e+00],
        [-4.94768569e+02, -3.81836609e+02, -6.12597656e+00],
        [ 4.94768569e+02,  3.81836609e+02, -6.12597656e+00],
        [ 4.23268569e+02,  3.81836609e+02, -6.12597656e+00],
        [ 4.94768554e+02, -7.62939453e-06,  1.25079102e+02],
        [ 4.94768569e+02

In [2]:
def define_global_coordinate_system(points):
    """
    Defines the global coordinate system by centering the given points at the origin.
    """
    global_origin = np.mean(points, axis=0)
    return points - global_origin, global_origin

def align_geometries(orig_X_Ia, orig_X_Ib, point_pairs):
    """
    Aligns geometry B to geometry A based on specified corresponding points.

    Parameters:
        orig_X_Ia (numpy.ndarray): (N,3) array of fixed geometry A's coordinates.
        orig_X_Ib (numpy.ndarray): (N,3) array of movable geometry B's coordinates.
        point_pairs (list of tuples): List of (index_B, index_A) pairs defining corresponding points.

    Returns:
        aligned_X_Ib (numpy.ndarray): (N,3) array of transformed B coordinates.
        R (numpy.ndarray): (3,3) rotation matrix.
        t (numpy.ndarray): (3,) translation vector.
    """
    if len(point_pairs) < 3:
        raise ValueError("At least 3 non-collinear point pairs are required for alignment.")

    # Define global coordinate system for both geometries
    X_Ia, global_origin_A = define_global_coordinate_system(orig_X_Ia.copy())
    X_Ib, _ = define_global_coordinate_system(orig_X_Ib.copy())

    # Extract corresponding points
    P_A = np.array([X_Ia[idx_A] for _, idx_A in point_pairs])
    P_B = np.array([X_Ib[idx_B] for idx_B, _ in point_pairs])

    # Compute centroids
    centroid_A = np.mean(P_A, axis=0)
    centroid_B = np.mean(P_B, axis=0)

    # Center the points
    Q_A = P_A - centroid_A
    Q_B = P_B - centroid_B

    # Compute covariance matrix
    H = Q_B.T @ Q_A

    # Compute SVD
    U, S, Vt = np.linalg.svd(H)

    # Compute rotation matrix
    R = Vt.T @ U.T

    # Ensure a proper rotation (no reflection)
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # Compute translation
    t = centroid_A - R @ centroid_B

    # Apply transformation to all points in B
    aligned_X_Ib = (R @ X_Ib.T).T + t + global_origin_A  # Reintroduce global origin

    return aligned_X_Ib, R, t


# Example Usage
# orig_X_Ia: Fixed geometry A (18 points, each with (x, y, z))
# orig_X_Ib: Movable geometry B (18 points, each with (x, y, z))

# Define corresponding points: (index_B, index_A)
point_pairs = [(5, 6), (7, 2), (1, 14)]  # B[5] -> A[6], B[1] -> A[14], B[0] -> A[13]

# Run alignment
aligned_X_Ib, R, t = align_geometries(orig_X_Ia, orig_X_Ib, point_pairs)

# Print results
print("Rotation Matrix R:\n", R)
print("Translation Vector t:\n", t)
print("Transformed B Coordinates:\n", aligned_X_Ib)


Rotation Matrix R:
 [[ 1.00000000e+00  1.67386208e-07 -7.63237098e-07]
 [-1.42815761e-07  9.99485319e-01  3.20795411e-02]
 [ 7.68213947e-07 -3.20795411e-02  9.99485319e-01]]
Translation Vector t:
 [494.76861396 383.4886177   -6.15245098]
Transformed B Coordinates:
 [[ 5.66268653e+02  3.81836755e+02 -6.12589392e+00]
 [ 4.23268653e+02  3.81836775e+02 -6.12600377e+00]
 [ 8.16268627e+02  7.71301389e+02  1.00323910e+02]
 [ 1.73268627e+02  7.71301481e+02  1.00323416e+02]
 [ 8.16268498e+02 -1.62820326e-04  1.25079651e+02]
 [ 1.73268498e+02 -7.09897918e-05  1.25079157e+02]
 [ 6.68037128e+02  3.85849220e+02  1.18888910e+02]
 [ 3.21499988e+02  3.85849270e+02  1.18888644e+02]
 [ 7.15001528e+01  7.63280392e+02 -2.44982408e+01]
 [ 1.52755178e-04  7.63280402e+02 -2.44982957e+01]
 [ 3.53298258e-06  3.85849308e+02  1.18888397e+02]
 [ 7.15000249e+01  2.22055153e-04  4.55255336e-05]
 [ 2.49268145e-05  2.32266480e-04 -9.40176366e-06]
 [ 9.89537291e+02  7.63280261e+02 -2.44975356e+01]
 [ 9.18037291e+02  7

In [3]:

def plot_geometries(geometries, name="3D Geometries"):
    """
    Plots multiple 3D geometries in a single k3d plot and labels the nodes.

    Parameters:
        geometries (list of tuples): Each tuple contains:
            - positions (numpy.ndarray): (N,3) array of vertex coordinates.
            - indices (numpy.ndarray): (M,3) array defining mesh faces.
            - color (int): Hex color for the geometry.
        name (str, optional): Name of the plot window.

    Returns:
        k3d.Plot: The k3d plot object.
    """
    plot = k3d.plot(name=name)

    for idx, (positions, indices, color) in enumerate(geometries):
        # Create the points object
        plt_points = k3d.points(positions=positions.astype(np.float32),
                                shader='3d',
                                color=color, 
                                point_size=30)
        
        # Create the mesh object
        plt_mesh = k3d.mesh(vertices=positions, 
                            indices=indices, 
                            color=color,  
                            opacity=0.5)
        
        # Add elements to the plot
        plot += plt_points
        plot += plt_mesh
        
        # Add labels for the nodes with slight offset
        for i, position in enumerate(positions):
            offset = np.array([0, 25 * (idx + 1), 0])  # Move text slightly in the z-direction
            label_position = position + offset
            
            label = k3d.text(
                text=str(i),  # Label is the index of the node
                position=label_position.astype(np.float32),
                color=color,
                size=1  # Reduce label size to prevent clutter
            )
            plot += label

    return plot



# Define colors0xff0000
color_A = 0xff0000  # Blue
color_B = 0x3f6bc5  # Red

# Plot list of geometries in one plot
plot = plot_geometries([
    (orig_X_Ia, orig_I_Fi, color_A),   # Geometry A
    (aligned_X_Ib, orig_I_Fi, color_B) # Geometry B
    ])

plot.display()


/Users/gui/Documents/bmcs_folder/bmcs_virtual/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/Users/gui/Documents/bmcs_folder/bmcs_virtual/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

In [4]:
orig_X_Ia[14], aligned_X_Ib[1]

(array([423.26856909, 381.83660889,  -6.12597656]),
 array([423.26865326, 381.8367752 ,  -6.12600377]))

In [5]:
orig_X_Ia[0], aligned_X_Ib[11]  # Check if the points are aligned

(array([71.5,  0. ,  0. ]),
 array([7.15000249e+01, 2.22055153e-04, 4.55255336e-05]))

In [6]:
orig_X_Ia[6], aligned_X_Ib[5]

(array([173.26856995,   0.        , 125.07910156]),
 array([ 1.73268498e+02, -7.09897918e-05,  1.25079157e+02]))

In [7]:
orig_X_Ia[2], aligned_X_Ib[7]

(array([321.5       , 385.84936523, 118.88867188]),
 array([321.49998785, 385.84926991, 118.88864378]))

In [8]:
import numpy as np

def slide_geometry(aligned_X_Ib, orig_X_Ia, point_start, point_end, d):
    """
    Slides geometry B along the vector from B[point_start] to B[point_end] by distance d.
    """
    # Compute direction vector from B[point_start] to B[point_end]
    v_B = aligned_X_Ib[point_end] - aligned_X_Ib[point_start]
    v_B_unit = v_B / np.linalg.norm(v_B)  # Normalize to unit vector

    # Compute translation vector
    t_B = d * v_B_unit

    # Apply translation
    moved_X_Ib = aligned_X_Ib + t_B

    return moved_X_Ib, v_B_unit

def compute_direction_vector(orig_X_Ia, p1, p2):
    """
    Compute the normalized direction vector for given points in geometry A.
    """
    v_A = orig_X_Ia[p2] - orig_X_Ia[p1]
    v_A_unit = v_A / np.linalg.norm(v_A)
    return v_A_unit

def check_colinearity(v_A_unit, v_B_unit, tolerance=1e-6):
    """
    Check if two vectors are colinear using the cross product method.
    """
    cross_product = np.cross(v_A_unit, v_B_unit)
    colinear = np.all(np.abs(cross_product) < tolerance)
    return colinear, cross_product

# Example usage
d = -57.1  # Define sliding distance
point_start_B, point_end_B = 1, 5  # Points in B
grid_start_A, grid_end_A = 6, 14  # Points in A

# Perform sliding operation
moved_X_Ib, v_B_unit = slide_geometry(aligned_X_Ib, orig_X_Ia, point_start_B, point_end_B, d)

# Compute direction vector for geometry A
v_A_unit = compute_direction_vector(orig_X_Ia, grid_start_A, grid_end_A)

# Check colinearity
colinear, cross_product = check_colinearity(v_A_unit, v_B_unit)

# Print results
print("Direction Vector for B:", v_B_unit)
print("Direction Vector for A:", v_A_unit)
print("Cross Product (should be close to 0 if colinear):", cross_product)
print("Are the vectors colinear?", colinear)


Direction Vector for B: [-0.52644534 -0.80406441  0.27628921]
Direction Vector for A: [ 0.52644534  0.80406441 -0.27628921]
Cross Product (should be close to 0 if colinear): [ 1.65675162e-09 -6.14518630e-10  1.36840883e-09]
Are the vectors colinear? True


In [9]:
# Plot both geometries in one plot
plot = plot_geometries([
    (orig_X_Ia, orig_I_Fi, color_A),   # Geometry A
    (moved_X_Ib, orig_I_Fi, color_B) # Geometry B
    ])

plot.display()

Output()

In [10]:
# compute the difference between node A=5 and B=6
orig_X_Ia[2][2] - moved_X_Ib[10][2]


15.776388681189673

In [11]:
import numpy as np

def compute_sliding_distance_z(orig_X_Ia, aligned_X_Ib, point_A, point_B, v_B_unit, t):
    """
    Compute the sliding distance d such that the z-coordinates satisfy:
    orig_X_Ia[point_A][2] - moved_X_Ib[point_B][2] = t
    """
    # Extract z-components
    delta_z = orig_X_Ia[point_A][2] - aligned_X_Ib[point_B][2] - t
    
    # Compute d using only the z-component
    d = delta_z / v_B_unit[2] if v_B_unit[2] != 0 else np.inf  # Avoid division by zero
    return d

def slide_geometry(aligned_X_Ib, v_B_unit, d):
    """
    Slides geometry B along its computed direction vector by distance d.
    """
    # Compute translation vector
    t_B = d * v_B_unit

    # Apply translation
    moved_X_Ib = aligned_X_Ib + t_B

    return moved_X_Ib

# Example usage
point_start_B, point_end_B = 1, 5  # Points in B
grid_start_A, grid_end_A = 6, 14  # Points in A
alignment_point_A, alignment_point_B = 2, 10  # Points to align
t = 50  # Target difference in z-coordinates

# Compute direction vector for B
v_B = aligned_X_Ib[point_end_B] - aligned_X_Ib[point_start_B]
v_B_unit = v_B / np.linalg.norm(v_B)  # Normalize to unit vector

# Compute sliding distance d based on z-coordinates
d = compute_sliding_distance_z(orig_X_Ia, aligned_X_Ib, alignment_point_A, alignment_point_B, v_B_unit, t)

# Apply sliding operation
moved_X_Ib = slide_geometry(aligned_X_Ib, v_B_unit, d)

# Print results
print("Computed Sliding Distance d:", d)
print("Moved Geometry B:\n", moved_X_Ib)


Computed Sliding Distance d: -180.96879461505677
Moved Geometry B:
 [[ 661.53883163  527.34732216  -56.12561909]
 [ 518.53883163  527.34734258  -56.12572894]
 [ 911.53880548  916.81195641   50.32418455]
 [ 268.53880548  916.81204824   50.32369059]
 [ 911.53867631  145.51040456   75.07992566]
 [ 268.53867631  145.51049639   75.0794317 ]
 [ 763.30730612  531.3597878    68.88918483]
 [ 416.77016622  531.35983729   68.88891861]
 [ 166.77033113  908.79095917  -74.49796599]
 [  95.27033113  908.79096938  -74.49802092]
 [  95.27018191  531.35987558   68.88867188]
 [ 166.7702033   145.51078944  -49.99967964]
 [  95.2702033   145.51079965  -49.99973457]
 [1084.80746932  908.79082806  -74.49726074]
 [1013.30746932  908.79083827  -74.49731567]
 [1084.80729043  531.35973426   68.88943205]
 [1084.80734149  145.51065833  -49.99897439]
 [1013.30734149  145.51066854  -49.99902932]]


In [12]:
# compute the difference between node A=5 and B=6
orig_X_Ia[2][2] - moved_X_Ib[10][2]

50.0

In [13]:
import numpy as np

def duplicate_and_offset_surface(orig_X_Ia, orig_I_Fi, t):
    """
    Duplicates a 3D surface geometry and offsets it in the z-direction by t.
    
    Parameters:
    - orig_X_Ia (numpy.ndarray): (N, 3) array of node coordinates.
    - orig_I_Fi (numpy.ndarray): (M, K) connectivity matrix (faces).
    - t (float): Offset distance in the z-direction.
    
    Returns:
    - new_X_Ia (numpay.ndarray): (2N, 3) array with both original and duplicated nodes.
    - new_I_Fi (numpy.ndarray): Updated (2M, K) connectivity matrix for the surface.
    """
    # Duplicate the nodes and apply the offset in the z-direction
    duplicated_X_Ia = orig_X_Ia.copy()
    duplicated_X_Ia[:, 2] += t  # Offset in the z-direction

    # Combine original and duplicated nodes
    new_X_Ia = np.vstack((orig_X_Ia, duplicated_X_Ia))

    # Update connectivity: new faces reference the offset nodes
    num_nodes = orig_X_Ia.shape[0]
    duplicated_I_Fi = orig_I_Fi + num_nodes  # Shift indices for duplicated nodes

    # Combine original and duplicated connectivity matrices
    new_I_Fi = np.vstack((orig_I_Fi, duplicated_I_Fi))

    return new_X_Ia, new_I_Fi

# Assuming orig_X_Ia is (18,3) and orig_I_Fi is (M,K)
# Define your orig_X_Ia and orig_I_Fi arrays before using this function

new_X_Ia, new_I_Fi = duplicate_and_offset_surface(orig_X_Ia, orig_I_Fi, t)

# Print results
print("New Node Coordinates (2N,3):\n", new_X_Ia)
print("New Connectivity Matrix (2M,K):\n", new_I_Fi)


New Node Coordinates (2N,3):
 [[ 7.15000000e+01  0.00000000e+00  0.00000000e+00]
 [-7.15000000e+01  0.00000000e+00  0.00000000e+00]
 [ 3.21500000e+02  3.85849365e+02  1.18888672e+02]
 [-3.21500000e+02  3.85849365e+02  1.18888672e+02]
 [ 3.21500000e+02 -3.85849365e+02  1.18888672e+02]
 [-3.21500000e+02 -3.85849365e+02  1.18888672e+02]
 [ 1.73268570e+02  0.00000000e+00  1.25079102e+02]
 [-1.73268570e+02  0.00000000e+00  1.25079102e+02]
 [-4.23268569e+02  3.81836609e+02 -6.12597656e+00]
 [-4.94768569e+02  3.81836609e+02 -6.12597656e+00]
 [-4.94768554e+02 -7.62939453e-06  1.25079102e+02]
 [-4.23268569e+02 -3.81836609e+02 -6.12597656e+00]
 [-4.94768569e+02 -3.81836609e+02 -6.12597656e+00]
 [ 4.94768569e+02  3.81836609e+02 -6.12597656e+00]
 [ 4.23268569e+02  3.81836609e+02 -6.12597656e+00]
 [ 4.94768554e+02 -7.62939453e-06  1.25079102e+02]
 [ 4.94768569e+02 -3.81836609e+02 -6.12597656e+00]
 [ 4.23268569e+02 -3.81836609e+02 -6.12597656e+00]
 [ 7.15000000e+01  0.00000000e+00  5.00000000e+01]
 

In [14]:
# Plot both geometries in one plot
plot = plot_geometries([
    (new_X_Ia, new_I_Fi, color_A)
    ])

plot.display()

Output()

In [15]:
# add list to the connectivity matrix to create a closed surface [34, 16, 15, 33]
new_I_Fi = np.append(new_I_Fi, [[34, 16, 33]], axis=0)

In [16]:
# Plot both geometries in one plot
plot = plot_geometries([
    (new_X_Ia, new_I_Fi, color_A)
    ])

plot.display()

Output()

In [17]:
def create_solid_connectivity(orig_I_Fi, num_nodes):
    """
    Generates the side faces to connect the original and offset surfaces.
    
    Parameters:
    - orig_I_Fi (numpy.ndarray): (M, K) connectivity matrix of the original surface.
    - num_nodes (int): Number of nodes in the original surface.
    
    Returns:
    - side_I_Fi (numpy.ndarray): Triangulated connectivity matrix for side faces.
    """
    side_faces = []

    for face in orig_I_Fi:
        if len(face) == 3:  # If the original faces are triangles
            a, b, c = face
            a2, b2, c2 = a + num_nodes, b + num_nodes, c + num_nodes

            # Create 3 quads (split into 6 triangles)
            side_faces.append([a, b, b2])
            side_faces.append([a, b2, a2])

            side_faces.append([b, c, c2])
            side_faces.append([b, c2, b2])

            side_faces.append([c, a, a2])
            side_faces.append([c, a2, c2])

    return np.array(side_faces, dtype=np.int32)

def duplicate_and_offset_surface_with_solid(orig_X_Ia, orig_I_Fi, t):
    """
    Duplicates a 3D surface, offsets it in the z-direction by t, and connects both layers to form a solid.
    
    Parameters:
    - orig_X_Ia (numpy.ndarray): (N,3) array of node coordinates.
    - orig_I_Fi (numpy.ndarray): (M,K) connectivity matrix (faces).
    - t (float): Offset distance in the z-direction.
    
    Returns:
    - new_X_Ia (numpy.ndarray): Updated node coordinates (2N,3).
    - solid_I_Fi (numpy.ndarray): Fully closed connectivity matrix.
    """
    num_nodes = orig_X_Ia.shape[0]

    # Duplicate nodes and apply offset in the z-direction
    offset_X_Ia = orig_X_Ia.copy()
    offset_X_Ia[:, 2] += t

    # Combine original and offset nodes
    new_X_Ia = np.vstack((orig_X_Ia, offset_X_Ia))

    # Compute connectivity for the offset surface
    offset_I_Fi = orig_I_Fi + num_nodes  # Shift indices for offset layer

    # Compute side faces to connect both layers
    side_I_Fi = create_solid_connectivity(orig_I_Fi, num_nodes)

    # **Flip the original faces to close the bottom**
    bottom_I_Fi = np.flip(orig_I_Fi, axis=1)  # Flip triangles/quads

    # Combine all faces
    solid_I_Fi = np.vstack((bottom_I_Fi, offset_I_Fi, side_I_Fi))

    return new_X_Ia, solid_I_Fi

import k3d
import numpy as np

def plot_solid_surfaces_in_k3d(geometries, colors=None):
    """
    Plots multiple 3D solid-like structures in k3d with node labels.
    
    Parameters:
    - geometries (list of tuples): Each tuple contains:
        - X_Ia (numpy.ndarray): (N,3) array of node coordinates.
        - I_Fi (numpy.ndarray): (M,K) connectivity matrix (faces).
    - colors (list of int, optional): List of hex colors for each geometry. If None, random colors are used.

    Returns:
    - k3d plot
    """
    plot = k3d.plot()

    num_geometries = len(geometries)
    if colors is None:
        colors = np.random.randint(0x1000000, 0xFFFFFF, size=num_geometries).tolist()

    for idx, (X_Ia, I_Fi) in enumerate(geometries):
        color = colors[idx % len(colors)]  # Assign color

        # Create the solid mesh (filled structure)
        mesh = k3d.mesh(X_Ia.astype(np.float32), I_Fi.astype(np.uint32),
                         color=color, wireframe=False, opacity=1.0, compression_level=9)
        plot += mesh

        # Add nodes as red points
        points = k3d.points(X_Ia, point_size=20, color=0xff0000)
        plot += points

        # Add node labels
        for i, position in enumerate(X_Ia):
            label = k3d.text(
                text=str(i),  # Node index as label
                position=position.astype(np.float32),
                color=0x000000,  # Black color for contrast
                size=0.7  # Adjust size to prevent clutter
            )
            plot += label

    return plot

# Generate the new geometry with a fully enclosed solid
new_X_Ia, solid_I_Fi = duplicate_and_offset_surface_with_solid(orig_X_Ia, orig_I_Fi, t)

geometries = [
    (new_X_Ia, solid_I_Fi),  # First geometry
]
# Plot in k3d with node numbers

plot = plot_solid_surfaces_in_k3d(geometries, colors=[0xe74c3c])
plot.display()

Output()

In [18]:
new_X_Ib = new_X_Ia.copy()

In [19]:
# orig_X_Ia: Fixed geometry A (18 points, each with (x, y, z))
# orig_X_Ib: Movable geometry B (18 points, each with (x, y, z))

# Define corresponding points: (index_B, index_A)
point_pairs = [(5, 24), (7, 20), (1, 32)]  # B[5] -> A[6], B[1] -> A[14], B[0] -> A[13]

# Run alignment
aligned_X_Ib, R, t = align_geometries(new_X_Ia, new_X_Ib, point_pairs)

# Print results
print("Rotation Matrix R:\n", R)
print("Translation Vector t:\n", t)
print("Transformed B Coordinates:\n", aligned_X_Ib)


Rotation Matrix R:
 [[ 1.00000000e+00  1.67386208e-07 -7.63237098e-07]
 [-1.42815761e-07  9.99485319e-01  3.20795411e-02]
 [ 7.68213947e-07 -3.20795411e-02  9.99485319e-01]]
Translation Vector t:
 [494.76859488 384.29060622  43.834682  ]
Transformed B Coordinates:
 [[ 5.66268653e+02  3.81836755e+02  4.38741061e+01]
 [ 4.23268653e+02  3.81836775e+02  4.38739962e+01]
 [ 8.16268627e+02  7.71301389e+02  1.50323910e+02]
 [ 1.73268627e+02  7.71301481e+02  1.50323416e+02]
 [ 8.16268498e+02 -1.62820326e-04  1.75079651e+02]
 [ 1.73268498e+02 -7.09897918e-05  1.75079157e+02]
 [ 6.68037128e+02  3.85849220e+02  1.68888910e+02]
 [ 3.21499988e+02  3.85849270e+02  1.68888644e+02]
 [ 7.15001528e+01  7.63280392e+02  2.55017592e+01]
 [ 1.52755178e-04  7.63280402e+02  2.55017043e+01]
 [ 3.53298258e-06  3.85849308e+02  1.68888397e+02]
 [ 7.15000249e+01  2.22055153e-04  5.00000455e+01]
 [ 2.49268145e-05  2.32266480e-04  4.99999906e+01]
 [ 9.89537291e+02  7.63280261e+02  2.55024644e+01]
 [ 9.18037291e+02  7

In [20]:
import k3d
import numpy as np

def plot_solid_surfaces_in_k3d(geometries, colors=None):
    """
    Plots multiple 3D solid-like structures in k3d with node labels and offset labels to prevent overlap.
    
    Parameters:
    - geometries (list of tuples): Each tuple contains:
        - X_Ia (numpy.ndarray): (N,3) array of node coordinates.
        - I_Fi (numpy.ndarray): (M,K) connectivity matrix (faces).
    - colors (list of int, optional): List of hex colors for each geometry. If None, random colors are used.

    Returns:
    - k3d plot
    """
    plot = k3d.plot()

    num_geometries = len(geometries)
    if colors is None:
        colors = np.random.randint(0, 0xFFFFFF, size=num_geometries).tolist()  # Fixed range

    for idx, (X_Ia, I_Fi) in enumerate(geometries):
        color = colors[idx % len(colors)]  # Assign color

        # Create the solid mesh (filled structure)
        mesh = k3d.mesh(X_Ia.astype(np.float32), I_Fi.astype(np.uint32),
                         color=color, wireframe=False, opacity=1.0, compression_level=9)
        plot += mesh

        # Add nodes as red points
        points = k3d.points(X_Ia, point_size=20, color=0xff0000)
        plot += points

        # Add node labels with offset to avoid overlap
        for i, position in enumerate(X_Ia):
            offset = np.array([25 * idx, 0, 0])  # Shift each geometry's labels in x-direction
            label_position = position + offset

            label = k3d.text(
                text=str(i),  # Node index as label
                position=label_position.astype(np.float32),
                color=color,  # Black color for contrast
                size=0.7  # Adjust size to prevent clutter
            )
            plot += label

    return plot

# Example usage
geometries = [
    (new_X_Ia, solid_I_Fi),  # First geometry
    (aligned_X_Ib, solid_I_Fi)   # Second geometry
]

plot = plot_solid_surfaces_in_k3d(geometries, colors=[0xe74c3c, 0x3498db])
plot.display()


Output()

In [21]:
import numpy as np

def find_sliding_distance(new_X_Ia, aligned_X_Ib, 
                          slide_dir_start=24, slide_dir_end=32, 
                          director_A_start=0, director_A_end=2, 
                          director_B_start=25, director_B_end=29):
    """
    Finds the sliding distance d along a direction defined by geometry A,
    so that after moving geometry B by d, the director vectors (A: 2-0, B: 29-25)
    become colinear.
    """
    # Direction along which geometry B slides
    u = new_X_Ia[slide_dir_end] - new_X_Ia[slide_dir_start]
    u = u / np.linalg.norm(u)

    # Director vector of geometry A
    v_A = new_X_Ia[director_A_end] - new_X_Ia[director_A_start]

    # Point on geometry B before displacement
    point_B = aligned_X_Ib[director_B_start]

    # Solve for d:
    # (point_B + d*u - point_A_start) = λ * v_A  --> rearranged to linear problem
    # We'll project difference vector onto direction perpendicular to v_A and solve for d.
    
    diff_vec = point_B - new_X_Ia[director_A_start]
    
    # We want: (point_B + d*u - point_A_start) to be parallel to v_A
    # That means the cross product with v_A should be zero
    # (point_B - point_A_start + d*u) x v_A = 0
    # Expand:
    # (diff_vec + d*u) x v_A = 0  --> (diff_vec x v_A) + d*(u x v_A) = 0
    # Solve for d:
    cross_diff = np.cross(diff_vec, v_A)
    cross_u = np.cross(u, v_A)
    
    # Solve component-wise: (cross_diff + d*cross_u) = 0  => d = -cross_diff / cross_u
    # We'll do a least-squares solution if not perfectly aligned:
    if np.linalg.norm(cross_u) < 1e-12:
        raise ValueError("The slide direction and director vector of A are parallel — cannot determine d.")
        
    d_solution, _, _, _ = np.linalg.lstsq(cross_u.reshape(-1, 1), -cross_diff, rcond=None)
    d = d_solution[0]
    
    print(f"Calculated sliding distance d: {d}")
    return d

def move_geometry_B(new_X_Ia, aligned_X_Ib, d, node_start=24, node_end=32):
    """
    Move geometry B along the direction defined by two nodes of geometry A by distance d.
    """
    direction_vec = new_X_Ia[node_end] - new_X_Ia[node_start]
    direction_unit = direction_vec / np.linalg.norm(direction_vec)
    displacement = d * direction_unit
    moved_X_Ib = aligned_X_Ib + displacement
    return moved_X_Ib


# Usage example with your loaded geometries:
d = find_sliding_distance(new_X_Ia, aligned_X_Ib,
                          slide_dir_start=24, slide_dir_end=32, 
                          director_A_start=0, director_A_end=2, 
                          director_B_start=25, director_B_end=29)

moved_X_Ib = move_geometry_B(new_X_Ia, aligned_X_Ib, d, node_start=24, node_end=32)


Calculated sliding distance d: 189.83321789961815


In [22]:
# Example usage
geometries = [
    (new_X_Ia, solid_I_Fi),  # First geometry
    (moved_X_Ib, solid_I_Fi)   # Second geometry
]

plot = plot_solid_surfaces_in_k3d(geometries, colors=[0xe74c3c, 0x3498db])
plot.display()

Output()

In [23]:
def export_to_obj(filename, X_Ia, I_Fi):
    """
    Exports the 3D geometry as an OBJ file.
    
    Parameters:
    - filename (str): Name of the output OBJ file.
    - X_Ia (numpy.ndarray): (N,3) array of node coordinates.
    - I_Fi (numpy.ndarray): (M,3) triangular connectivity matrix (1-based indices for OBJ format).
    """
    with open(filename, "w") as f:
        # Write vertices
        for v in X_Ia:
            f.write(f"v {v[0]} {v[1]} {v[2]}\n")

        # Write faces (OBJ format uses 1-based indexing)
        for face in I_Fi:
            f.write(f"f {face[0] + 1} {face[1] + 1} {face[2] + 1}\n")

    print(f"OBJ file saved as: {filename}")

# Example usage:
export_to_obj("solid_geometry.obj", new_X_Ia, solid_I_Fi)





OBJ file saved as: solid_geometry.obj
